In [15]:
import pandas as pd 
import os 
df_in = pd.read_csv('gdc_sample_sheet.2020-06-16.tsv', sep='\t')
df_in = df_in[df_in['Project ID'] == 'TCGA-GBM']


In [16]:
def getSampleSourceFromSampleId(sample_id):
    '''
    No more in use. 
    '''
    source_int = int(sample_id.split('-')[3][:2])
    if source_int < 10:
        source = 'tumuor'
    elif source_int == 10:
        source = 'blood'
    else:
        source = 'normal'
    return source 

def getFileTypeSuffix(file_name):
    '''
    Only get the first content before the first . out. 
    '''
    return '.'.join(file_name.split('.')[1:])

df_in.loc[:,'FileSuffix'] = df_in.loc[:,'File Name'].map(getFileTypeSuffix)

# Deseq2 request raw data. 

df_in = df_in[df_in['FileSuffix'] == 'htseq.counts.gz']
df_in['Sample Type'].value_counts()




Primary Tumor          156
Recurrent Tumor         13
Solid Tissue Normal      5
Name: Sample Type, dtype: int64

In [17]:
df_in.drop(index=df_in[df_in['Sample Type']=='Recurrent Tumor'].index,inplace=True)
df_in.drop_duplicates(subset=['Sample ID'], inplace=True)

In [18]:
df_in['Sample Type'].value_counts()

Primary Tumor          155
Solid Tissue Normal      5
Name: Sample Type, dtype: int64

In [19]:
# Read dfs from each folder. 
import os 
dfs = [pd.read_csv(os.path.join(folder, filename), sep='\t', index_col=0, header=None) for folder,filename in zip(df_in['File ID'],df_in['File Name'])]
[df.drop(index=filter(lambda x:x.startswith('__'), df.index), inplace=True) for df in dfs]

for sample_id, df in zip(df_in['Sample ID'], dfs):
    df.columns = [sample_id]

# dfs_without_dup = []
# sample_ids = []
# for sample_id,df in zip(df_in['Sample ID'], dfs):
#     if sample_id in sample_ids:
#         continue
#     else:
#         dfs_without_dup.append(df)

# dfs = dfs_without_dup


In [20]:
df_concated_raw_counts = pd.concat(dfs, axis=1)
df_concated_raw_counts.to_csv('01.inputdata.txt', sep='\t')

In [21]:
df_clinical = df_in.loc[:,['Sample ID', 'Sample Type']]
df_clinical.replace({'Primary Tumor':'T', 'Solid Tissue Normal':'N'}, inplace=True)
df_clinical.rename(columns={'Sample ID':'sample_id', 'Sample Type':'condition'},inplace=True)
df_clinical.drop_duplicates(inplace=True)
df_clinical.to_csv('01.coldata.txt', sep='\t', index=False)